In [3]:
import pandas
import numpy as np

In [4]:
data_set = pandas.read_excel('./TJ+Admission+Data+Classes+of+2017+and+2018.xlsx')

In [5]:
del data_set["ID"]
del data_set["Application Year"]
del data_set["Semifinalist"]
del data_set["CombineScore"]
del data_set["Math and Verbal"]
del data_set["M/S GPA"]

In [6]:
data_set = data_set.fillna(0)
data_set.head()

,FC Resident,AAP,Gender,GPA,Verbal Score,Math Score,Final Decision
0,Yes,Yes,F,3.9429,42.0,45.0,N
1,Yes,Yes,M,3.8600,35.0,20.0,0
2,Yes,Yes,M,4.0000,46.0,30.0,N
3,Yes,Yes,M,3.3143,21.0,20.0,0
4,Yes,Yes,M,3.1450,27.0,34.0,0


In [7]:
len(data_set)

6072

In [8]:
def label_FC_AAP(row):
    if row['FC Resident'] == "Yes":
        if row['AAP'] == "Yes":
            return 2
        else:
            return 1
    else:
        return 0

In [9]:
data_set['FC/AAP'] = data_set.apply(lambda row: label_FC_AAP(row),axis=1)
    

In [10]:
data_set.tail()

,FC Resident,AAP,Gender,GPA,Verbal Score,Math Score,Final Decision,FC/AAP
6067,Yes,0,M,3.7800,38.0,35.0,N,1
6068,Yes,0,M,3.9714,36.0,31.0,N,1
6069,Yes,0,F,4.0000,27.0,18.0,0,1
6070,No,0,M,3.6667,23.0,26.0,0,0
6071,No,0,F,4.0000,34.0,37.0,N,0


In [11]:
cols = data_set.columns.tolist()
cols = cols[-1:] + cols[:-1]
data_set = data_set[cols]

In [12]:
data_set.head()

,FC/AAP,FC Resident,AAP,Gender,GPA,Verbal Score,Math Score,Final Decision
0,2,Yes,Yes,F,3.9429,42.0,45.0,N
1,2,Yes,Yes,M,3.8600,35.0,20.0,0
2,2,Yes,Yes,M,4.0000,46.0,30.0,N
3,2,Yes,Yes,M,3.3143,21.0,20.0,0
4,2,Yes,Yes,M,3.1450,27.0,34.0,0


In [13]:
del data_set['FC Resident']
del data_set['AAP']
data_set.head()

,FC/AAP,Gender,GPA,Verbal Score,Math Score,Final Decision
0,2,F,3.9429,42.0,45.0,N
1,2,M,3.8600,35.0,20.0,0
2,2,M,4.0000,46.0,30.0,N
3,2,M,3.3143,21.0,20.0,0
4,2,M,3.1450,27.0,34.0,0


In [14]:
np_data = data_set.as_matrix()

In [15]:
np_data[np_data == 'Yes'] = 1
np_data[np_data == 'Y'] = 2
np_data[np_data == 'W'] = 1 
np_data[np_data == 'N'] = 0
np_data[np_data == 'No'] = 0
np_data[np_data == "F"] = 0
np_data[np_data == "M"] = 1

In [16]:
print(data_set.head())
all_data = np_data
np.random.shuffle(all_data)
# Y = np_data[:, 6].astype(float)
# X = np_data[:, [i for i in range(6)]].astype(float)
Y = all_data[:, 5].astype(float)
X = np_data[:, [i for i in range(5)]].astype(float)

print(Y.shape)
y_train, y_test = Y[:5000], Y[5000:]
x_train, x_test = X[:5000,:], X[5000:,:]
print(x_train.shape)
print(y_train.shape)

   FC/AAP Gender     GPA  Verbal Score  Math Score Final Decision
0       2      F  3.9429          42.0        45.0              N
1       2      M  3.8600          35.0        20.0              0
2       2      M  4.0000          46.0        30.0              N
3       2      M  3.3143          21.0        20.0              0
4       2      M  3.1450          27.0        34.0              0
(6072,)
(5000, 5)
(5000,)


## Keras Classification

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import losses
from keras import optimizers
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [25]:
np.random.seed(1)

y_binary = to_categorical(Y)
y_test_binary = to_categorical(y_test)
y_train_binary = to_categorical(y_train)

In [32]:
model = Sequential()
model.add(Dense(32, input_dim=5, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
model.add(Dense(32, kernel_initializer='normal', activation='relu'))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.10))
model.add(Dense(3, activation='softmax'))


In [33]:
adam = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-05, decay=0.001)
# adam = optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# sgd = optimizers.SGD(lr=0.1, decay=1e-8, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=sgd)

In [34]:
model.fit(x_train, y_train_binary, batch_size=500, epochs=1000, validation_split=0.2, shuffle=True)

Train on 4000 samples, validate on 1000 samples
Epoch 1/1000
4000/4000 [==============================] - 0s - loss: 0.8506 - acc: 0.7380 - val_loss: 0.6579 - val_acc: 0.8300
Epoch 2/1000
4000/4000 [==============================] - 0s - loss: 0.6388 - acc: 0.8292 - val_loss: 0.6185 - val_acc: 0.8300
Epoch 3/1000
4000/4000 [==============================] - 0s - loss: 0.6036 - acc: 0.8293 - val_loss: 0.5806 - val_acc: 0.8300
Epoch 4/1000
4000/4000 [==============================] - 0s - loss: 0.5643 - acc: 0.8293 - val_loss: 0.5459 - val_acc: 0.8300
Epoch 5/1000
4000/4000 [==============================] - 0s - loss: 0.5244 - acc: 0.8293 - val_loss: 0.4892 - val_acc: 0.8300
Epoch 6/1000
4000/4000 [==============================] - 0s - loss: 0.4615 - acc: 0.8293 - val_loss: 0.4145 - val_acc: 0.8300
Epoch 7/1000
4000/4000 [==============================] - 0s - loss: 0.3902 - acc: 0.8293 - val_loss: 0.3503 - val_acc: 0.8300
Epoch 8/1000
4000/4000 [==============================] - 0s - 

4000/4000 [==============================] - 0s - loss: 0.2980 - acc: 0.8728 - val_loss: 0.2977 - val_acc: 0.8720
Epoch 65/1000
4000/4000 [==============================] - 0s - loss: 0.2979 - acc: 0.8757 - val_loss: 0.2934 - val_acc: 0.8770
Epoch 66/1000
4000/4000 [==============================] - 0s - loss: 0.2978 - acc: 0.8735 - val_loss: 0.2940 - val_acc: 0.8780
Epoch 67/1000
4000/4000 [==============================] - 0s - loss: 0.2996 - acc: 0.8765 - val_loss: 0.2929 - val_acc: 0.8750
Epoch 68/1000
4000/4000 [==============================] - 0s - loss: 0.3009 - acc: 0.8710 - val_loss: 0.2951 - val_acc: 0.8760
Epoch 69/1000
4000/4000 [==============================] - 0s - loss: 0.3037 - acc: 0.8685 - val_loss: 0.2915 - val_acc: 0.8830
Epoch 70/1000
4000/4000 [==============================] - 0s - loss: 0.2981 - acc: 0.8747 - val_loss: 0.2932 - val_acc: 0.8750
Epoch 71/1000
4000/4000 [==============================] - 0s - loss: 0.2971 - acc: 0.8763 - val_loss: 0.3045 - val_ac

4000/4000 [==============================] - 0s - loss: 0.2898 - acc: 0.8825 - val_loss: 0.2845 - val_acc: 0.8830
Epoch 128/1000
4000/4000 [==============================] - 0s - loss: 0.2876 - acc: 0.8788 - val_loss: 0.2846 - val_acc: 0.8820
Epoch 129/1000
4000/4000 [==============================] - 0s - loss: 0.2893 - acc: 0.8773 - val_loss: 0.2844 - val_acc: 0.8830
Epoch 130/1000
4000/4000 [==============================] - 0s - loss: 0.2917 - acc: 0.8772 - val_loss: 0.2972 - val_acc: 0.8740
Epoch 131/1000
4000/4000 [==============================] - 0s - loss: 0.2995 - acc: 0.8737 - val_loss: 0.2864 - val_acc: 0.8810
Epoch 132/1000
4000/4000 [==============================] - 0s - loss: 0.2875 - acc: 0.8838 - val_loss: 0.2827 - val_acc: 0.8840
Epoch 133/1000
4000/4000 [==============================] - 0s - loss: 0.2876 - acc: 0.8822 - val_loss: 0.2822 - val_acc: 0.8840
Epoch 134/1000
4000/4000 [==============================] - 0s - loss: 0.2930 - acc: 0.8793 - val_loss: 0.2826 -

4000/4000 [==============================] - 0s - loss: 0.2769 - acc: 0.8855 - val_loss: 0.2772 - val_acc: 0.8850
Epoch 191/1000
4000/4000 [==============================] - 0s - loss: 0.2771 - acc: 0.8870 - val_loss: 0.2731 - val_acc: 0.8870
Epoch 192/1000
4000/4000 [==============================] - 0s - loss: 0.2739 - acc: 0.8898 - val_loss: 0.2745 - val_acc: 0.8850
Epoch 193/1000
4000/4000 [==============================] - 0s - loss: 0.2793 - acc: 0.8820 - val_loss: 0.2742 - val_acc: 0.8860
Epoch 194/1000
4000/4000 [==============================] - 0s - loss: 0.2751 - acc: 0.8905 - val_loss: 0.2737 - val_acc: 0.8880
Epoch 195/1000
4000/4000 [==============================] - 0s - loss: 0.2799 - acc: 0.8852 - val_loss: 0.2878 - val_acc: 0.8760
Epoch 196/1000
4000/4000 [==============================] - 0s - loss: 0.2759 - acc: 0.8895 - val_loss: 0.2760 - val_acc: 0.8820
Epoch 197/1000
4000/4000 [==============================] - 0s - loss: 0.2747 - acc: 0.8885 - val_loss: 0.2728 -

4000/4000 [==============================] - 0s - loss: 0.2744 - acc: 0.8845 - val_loss: 0.2845 - val_acc: 0.8750
Epoch 254/1000
4000/4000 [==============================] - 0s - loss: 0.2724 - acc: 0.8923 - val_loss: 0.2738 - val_acc: 0.8850
Epoch 255/1000
4000/4000 [==============================] - 0s - loss: 0.2681 - acc: 0.8877 - val_loss: 0.2675 - val_acc: 0.8880
Epoch 256/1000
4000/4000 [==============================] - 0s - loss: 0.2693 - acc: 0.8890 - val_loss: 0.2719 - val_acc: 0.8880
Epoch 257/1000
4000/4000 [==============================] - 0s - loss: 0.2683 - acc: 0.8902 - val_loss: 0.2683 - val_acc: 0.8890
Epoch 258/1000
4000/4000 [==============================] - 0s - loss: 0.2673 - acc: 0.8897 - val_loss: 0.2686 - val_acc: 0.8880
Epoch 259/1000
4000/4000 [==============================] - 0s - loss: 0.2664 - acc: 0.8935 - val_loss: 0.2704 - val_acc: 0.8870
Epoch 260/1000
4000/4000 [==============================] - 0s - loss: 0.2666 - acc: 0.8925 - val_loss: 0.2734 -

...

4000/4000 [==============================] - 0s - loss: 0.2590 - acc: 0.8958 - val_loss: 0.2681 - val_acc: 0.8870
Epoch 695/1000
4000/4000 [==============================] - 0s - loss: 0.2607 - acc: 0.8940 - val_loss: 0.2674 - val_acc: 0.8860
Epoch 696/1000
4000/4000 [==============================] - 0s - loss: 0.2574 - acc: 0.8955 - val_loss: 0.2642 - val_acc: 0.8840
Epoch 697/1000
4000/4000 [==============================] - 0s - loss: 0.2566 - acc: 0.8948 - val_loss: 0.2652 - val_acc: 0.8860
Epoch 698/1000
4000/4000 [==============================] - 0s - loss: 0.2568 - acc: 0.8945 - val_loss: 0.2641 - val_acc: 0.8860
Epoch 699/1000
4000/4000 [==============================] - 0s - loss: 0.2565 - acc: 0.8953 - val_loss: 0.2627 - val_acc: 0.8840
Epoch 700/1000
4000/4000 [==============================] - 0s - loss: 0.2565 - acc: 0.8950 - val_loss: 0.2654 - val_acc: 0.8860
Epoch 701/1000
4000/4000 [==============================] - 0s - loss: 0.2564 - acc: 0.8955 - val_loss: 0.2662 -

4000/4000 [==============================] - 0s - loss: 0.2576 - acc: 0.8957 - val_loss: 0.2643 - val_acc: 0.8830
Epoch 758/1000
4000/4000 [==============================] - 0s - loss: 0.2558 - acc: 0.8938 - val_loss: 0.2641 - val_acc: 0.8830
Epoch 759/1000
4000/4000 [==============================] - 0s - loss: 0.2558 - acc: 0.8965 - val_loss: 0.2644 - val_acc: 0.8860
Epoch 760/1000
4000/4000 [==============================] - 0s - loss: 0.2557 - acc: 0.8953 - val_loss: 0.2654 - val_acc: 0.8810
Epoch 761/1000
4000/4000 [==============================] - 0s - loss: 0.2556 - acc: 0.8945 - val_loss: 0.2645 - val_acc: 0.8800
Epoch 762/1000
4000/4000 [==============================] - 0s - loss: 0.2564 - acc: 0.8945 - val_loss: 0.2682 - val_acc: 0.8830
Epoch 763/1000
4000/4000 [==============================] - 0s - loss: 0.2561 - acc: 0.8963 - val_loss: 0.2652 - val_acc: 0.8830
Epoch 764/1000
4000/4000 [==============================] - 0s - loss: 0.2566 - acc: 0.8955 - val_loss: 0.2638 -

4000/4000 [==============================] - 0s - loss: 0.2558 - acc: 0.8938 - val_loss: 0.2649 - val_acc: 0.8830
Epoch 821/1000
4000/4000 [==============================] - 0s - loss: 0.2547 - acc: 0.8958 - val_loss: 0.2682 - val_acc: 0.8830
Epoch 822/1000
4000/4000 [==============================] - 0s - loss: 0.2563 - acc: 0.8955 - val_loss: 0.2651 - val_acc: 0.8840
Epoch 823/1000
4000/4000 [==============================] - 0s - loss: 0.2546 - acc: 0.8947 - val_loss: 0.2657 - val_acc: 0.8810
Epoch 824/1000
4000/4000 [==============================] - 0s - loss: 0.2552 - acc: 0.8940 - val_loss: 0.2665 - val_acc: 0.8800
Epoch 825/1000
4000/4000 [==============================] - 0s - loss: 0.2561 - acc: 0.8940 - val_loss: 0.2726 - val_acc: 0.8880
Epoch 826/1000
4000/4000 [==============================] - 0s - loss: 0.2565 - acc: 0.8948 - val_loss: 0.2648 - val_acc: 0.8840
Epoch 827/1000
4000/4000 [==============================] - 0s - loss: 0.2552 - acc: 0.8938 - val_loss: 0.2656 -

4000/4000 [==============================] - 0s - loss: 0.2558 - acc: 0.8945 - val_loss: 0.2710 - val_acc: 0.8830
Epoch 884/1000
4000/4000 [==============================] - 0s - loss: 0.2570 - acc: 0.8932 - val_loss: 0.2688 - val_acc: 0.8800
Epoch 885/1000
4000/4000 [==============================] - 0s - loss: 0.2552 - acc: 0.8965 - val_loss: 0.2675 - val_acc: 0.8840
Epoch 886/1000
4000/4000 [==============================] - 0s - loss: 0.2555 - acc: 0.8960 - val_loss: 0.2642 - val_acc: 0.8830
Epoch 887/1000
4000/4000 [==============================] - 0s - loss: 0.2539 - acc: 0.8947 - val_loss: 0.2642 - val_acc: 0.8850
Epoch 888/1000
4000/4000 [==============================] - 0s - loss: 0.2540 - acc: 0.8950 - val_loss: 0.2659 - val_acc: 0.8830
Epoch 889/1000
4000/4000 [==============================] - 0s - loss: 0.2540 - acc: 0.8963 - val_loss: 0.2652 - val_acc: 0.8830
Epoch 890/1000
4000/4000 [==============================] - 0s - loss: 0.2540 - acc: 0.8953 - val_loss: 0.2661 -

4000/4000 [==============================] - 0s - loss: 0.2558 - acc: 0.8973 - val_loss: 0.2802 - val_acc: 0.8840
Epoch 947/1000
4000/4000 [==============================] - 0s - loss: 0.2554 - acc: 0.8970 - val_loss: 0.2657 - val_acc: 0.8840
Epoch 948/1000
4000/4000 [==============================] - 0s - loss: 0.2558 - acc: 0.8958 - val_loss: 0.2703 - val_acc: 0.8850
Epoch 949/1000
4000/4000 [==============================] - 0s - loss: 0.2559 - acc: 0.8930 - val_loss: 0.2649 - val_acc: 0.8820
Epoch 950/1000
4000/4000 [==============================] - 0s - loss: 0.2553 - acc: 0.8982 - val_loss: 0.2740 - val_acc: 0.8850
Epoch 951/1000
4000/4000 [==============================] - 0s - loss: 0.2593 - acc: 0.8947 - val_loss: 0.2755 - val_acc: 0.8830
Epoch 952/1000
4000/4000 [==============================] - 0s - loss: 0.2554 - acc: 0.8983 - val_loss: 0.2666 - val_acc: 0.8820
Epoch 953/1000
4000/4000 [==============================] - 0s - loss: 0.2551 - acc: 0.8960 - val_loss: 0.2699 -

In [37]:
scores = model.evaluate(X, y_binary)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

4192/6072 [===================>..........] - ETA: 0s
acc: 89.10%


In [44]:
#FC/AAP (2 for both, 1 for FC, 0 for neither), Gender(F==0), GPA,  Verbal Score,  Math Score
model.predict(np.array([[2, 0, 4.0, 48, 48]]))
#Output: no, waitlist, yes

array([[ 0.21934761,  0.05461366,  0.72603875]], dtype=float32)

In [33]:
qw = np.array([[1, 1, 1, 3.0, 40, 40]])
qw.shape

(1, 6)

In [38]:
l_count = 0
m_count = 0
w_count = 0
for x_ in X:
#     print(x_)
    res = model.predict(np.array([x_]))
#     print(res)
#     print(res[0])
    results = res[0]
    if results[0] > results[1] and results[0] > results[2]:
#         print("L")
        l_count +=1
    elif results[1] > results[0] and results[1] > results[2]:
#         print("M")
        m_count +=1
    elif results[2] > results[0] and results[2] > results[1]:
#         print("W")
#         print(x_)
        w_count +=1
    
print(l_count, m_count, w_count)

5249 0 823


In [ ]:
from sklearn.neural_network import MLPClassifier

In [113]:
clf = MLPClassifier(solver='adam', activation='tanh', epsilon=1e-7, alpha=1e-5, batch_size=500, tol=1e-8, momentum=0.9, learning_rate_init=0.01, hidden_layer_sizes=(100), max_iter=200, verbose=True)

In [114]:
clf.fit(x_train, y_train)

Iteration 1, loss = 0.65881056
Iteration 2, loss = 0.53996848
Iteration 3, loss = 0.52111229
Iteration 4, loss = 0.51463119
Iteration 5, loss = 0.51356468
Iteration 6, loss = 0.51106848
Iteration 7, loss = 0.51258082
Iteration 8, loss = 0.51228424
Iteration 9, loss = 0.51177624
Training loss did not improve more than tol=0.000000 for two consecutive epochs. Stopping.


MLPClassifier(activation='tanh', alpha=1e-05, batch_size=500, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-07,
       hidden_layer_sizes=100, learning_rate='constant',
       learning_rate_init=0.01, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=1e-08, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [115]:
clf.score(x_test, y_test)

0.83488805970149249